In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
def combinations(l):
  res = []
  for i in range(len(l)):
    for j in range(i+1, len(l)):
      res.append((l[i], l[j]))
      res.append((l[j], l[i]))
  return res

def combinations_filtered(l):
  res = []
  for i in range(len(l)):
    for j in range(i+1, len(l)):
      if l[i] in filter_list or l[j] in filter_list:
        res.append((l[i], l[j]))
        res.append((l[j], l[i]))
  return res

def find_follow_pairs(user_data):
  if len(user_data.split()) != 2:
    return []
  user = user_data.split()[0]
  user_followers = user_data.split()[1].split(',')
  follow_pairs = []
  for i in user_followers:
    follow_pairs.append((user, i))
  return follow_pairs

def find_mutually_follow_pairs(user_data):
  if len(user_data.split()) != 2:
    return []
  user = user_data.split()[0]
  user_followers = user_data.split()[1].split(',')
  follow_pairs = []
  for i in combinations(user_followers):
  #for i in combinations_filtered(user_followers):
    follow_pairs.append(i)
  return follow_pairs

def output_convertor(inp):
  u, r = inp
  res = u + '\t'
  for i in r:
    res += i
    res += ','
  return res[:-1]

In [ ]:
adj_file = sc.textFile('input1.txt', 1)

In [ ]:
follow_pairs = adj_file.flatMap(find_follow_pairs).map(lambda a:(a, -1 * np.inf))
mutually_followed_pairs = adj_file.flatMap(find_mutually_follow_pairs).map(lambda a:(a, 1))
combined_pairs = follow_pairs.union(mutually_followed_pairs)
recommendations = combined_pairs.reduceByKey(lambda a,b: a+b).filter(lambda a: a[1] > 0).map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey()
recommendations_sorted = recommendations.map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0],  sorted(x[1], reverse = True, key = lambda a: a[1])))
recommendations_sorted = recommendations_sorted.map(lambda x: (x[0], [i[0] for i in x[1]]))

In [ ]:
#recommendations_sorted.filter(lambda x: x[0] in filter_list).collect()

In [ ]:
recommendations_sorted1 = recommendations_sorted.map(output_convertor)

In [ ]:
recommendations_sorted1.collect()

In [ ]:
recommendations_sorted1.coalesce(1).saveAsTextFile("res.txt")